# 512×512 Tile Preprocessing with Progress
Slices original images and masks into 512×512 tiles, discarding empty masks, and shows progress bars.

In [ ]:
import os
from PIL import Image
import numpy as np
from tqdm import tqdm

# Configuration
orig_parent = 'Mikroplastikai'
orig_img_dir  = os.path.join(orig_parent, 'images')
orig_mask_dir = os.path.join(orig_parent, 'masks')

new_parent = '512'
new_img_dir  = os.path.join(new_parent, 'images')
new_mask_dir = os.path.join(new_parent, 'masks')

tile_size = 512

# Create output folders
os.makedirs(new_img_dir, exist_ok=True)
os.makedirs(new_mask_dir, exist_ok=True)

# Get list of images
img_names = [f for f in os.listdir(orig_img_dir)
             if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif'))]

# Process with progress
for img_name in tqdm(img_names, desc='Images'):
    base, _ = os.path.splitext(img_name)
    img_path = os.path.join(orig_img_dir, img_name)

    # Find corresponding mask
    mask_path = None
    for ext in ('.png', '.jpg', '.jpeg', '.tif'):
        candidate = os.path.join(orig_mask_dir, base + ext)
        if os.path.exists(candidate):
            mask_path = candidate
            break
    if mask_path is None:
        tqdm.write(f"Warning: No mask for {img_name}, skipped.")
        continue

    img = Image.open(img_path).convert('RGB')
    mask = Image.open(mask_path).convert('L')

    w, h = img.size
    nx = w // tile_size
    ny = h // tile_size

    # Iterate tiles
    for i in range(ny):
        for j in range(nx):
            left, upper = j * tile_size, i * tile_size
            right, lower = left + tile_size, upper + tile_size

            tile_img = img.crop((left, upper, right, lower))
            tile_mask = mask.crop((left, upper, right, lower))

            arr = np.array(tile_mask)
            if arr.max() == 0:
                continue

            tile_fname = f"{base}_r{i}_c{j}.png"
            tile_img.save(os.path.join(new_img_dir, tile_fname))
            tile_mask.save(os.path.join(new_mask_dir, tile_fname))

tqdm.write(f"Done. Tiles saved to '{new_parent}'.")